# Setting up your development environ

1. Clone the state scraper repo from Github

  ```
  git clone https://github.com/influence-usa/scrapers-us-state.git
  ```

2. Make a new virtualenv

  ```
  mkvirtualenv --python=$(which python3)
  ```
  
3. Use `pip` to install requirements

  ```
  pip install -r requirements.txt
  ```
  
4. If you don't see a folder for the state you're working on, run the following:

  ```
    (iusa-scrape)$>pupa init arizona
    no pupa_settings on path, using defaults
    jurisdiction name (e.g. City of Seattle): Arizona
    division id (e.g. ocd-division/country:us/state:wa/place:seattle): ocd-division/country:us/state:az
    classification (can be: government, legislature, executive, school_system): government
    official URL: http://www.az.gov/
    create disclosures scraper? [Y/n]: Y
    create bills scraper? [y/N]: n
    create events scraper? [y/N]: y
    create votes scraper? [y/N]: n
    create people scraper? [y/N]: y
  ```

...what this did was create a new folder for the state. In this example, the state was Arizona (`arizona`).

  ```
    (iusa-scrape)$>tree
    .
    ├── ak
    │   └── __init__.py
    ├── al
    │   ├── __init__.py
    │   └── people.py
    ├── arizona
    │   ├── disclosures.py
    │   ├── events.py
    │   ├── __init__.py
    │   └── people.py
    ├── md
    │   └── __init__.py
    ├── README.md
    ├── requirements.txt
    ├── Untitled.ipynb
    └── utils
        ├── __init__.py
        └── lxmlize.py
  ```
  
To follow the broader pupa convention, we'll change the directory name to `az`:

```
    (iusa-scrape)$>mv arizona az
    (iusa-scrape)$>tree
    tree
    .
    ├── ak
    │   └── __init__.py
    ├── al
    │   ├── __init__.py
    │   └── people.py
    ├── az
    │   ├── disclosures.py
    │   ├── events.py
    │   ├── __init__.py
    │   └── people.py
    ├── md
    │   └── __init__.py
    ├── README.md
    ├── requirements.txt
    ├── Untitled.ipynb
    └── utils
        ├── __init__.py
        └── lxmlize.py

5 directories, 13 files
```
  
Because we told it to in teh questions asked above, it also created the starter code for our scrapers: there's one each for disclosures, events and people.

Also interesting is the `__init__.py` file in our state's directory.  It used the answers to our questions to build a `Jurisdiction` object that represents the state government:

```
class Arizona(Jurisdiction):
    division_id = "ocd-division/country:us/state:az"
    classification = "government"
    name = "Arizona"
    url = "https://az.gov/"
    scrapers = {
        "events": ArizonaEventScraper,
        "people": ArizonaPersonScraper,
        "disclosures": ArizonaDisclosureScraper,
    }

    def get_organizations(self):
        yield Organization(name=None, classification=None)
```

# Version control

This is a good time to add and commit our changes so far.

```
(iusa-scrape)$>git add az
(iusa-scrape)$>git commit -m "initialized arizona"
[master 3e622ef] initialized arizona
 4 files changed, 47 insertions(+)
 create mode 100644 az/__init__.py
 create mode 100644 az/disclosures.py
 create mode 100644 az/events.py
 create mode 100644 az/people.py
 ```

# Where is the data?

# Creating global authority organizations

## Create the Secretary of State

The `get_organizations` method of the `Jurisdiction` class lets us define some global organizations for all of the data that we'll be scraping from Arizona's sites. For campaign finance disclosures, we'll have to define the Arizona Secretary of State's Office.

```
def get_organizations(self):                                        
```

First, initialize using the `Organization` class.

```
    secretary_of_state = Organization(                                    
        name="Office of the Secretary of State, State of Arizona",        
        classification="office"                                           
    )                                                               ```
    
Here, we're able to set particular attributes using `kwargs`.  To get a sense of which attributes you can set at this point, check out the [source](https://github.com/influence-usa/pupa/blob/disclosures/pupa/scrape/popolo.py#L132-L182).

Now, we can add other attribtues, using the helper methods found on the `Organization` class:

```
    secretary_of_state.add_contact_detail(                                
        type="voice",                                                     
        value="602-542-4285"                                              
    )                    

    secretary_of_state.add_contact_detail(                                
        type="address",                                                   
        value="1700 W Washington St Fl 7, Phoenix AZ 85007-2808"          
    )                                                                     
    secretary_of_state.add_link(                                          
        url="http://www.azsos.gov/",                                      
        note="Home page"                                                  
    )                                                                     
```

We should add the organization we've created to the `Jurisdiction` object as a semi-private property. This is useful, beacuse the `Jurisdiction` object will essentially always be accessible to all of our scrapers. Whenever we want to refer to the AZ Secretary of State, we can always access it from `Arizona` jurisdiction object.

```
    self._secretary_of_state = secretary_of_state                   
```

Finally, yield the organization we created. This is beacause `get_organizations` is actually the first scraper that we'll run each time we run Arizona scrapers of any kind.

```
    yield secretary_of_state                                          
```

## Test what we have so far!

Cool, let's try out what we have so far.  From the project root (`scrapers-us-state`), run the command:

```
(iusa-scrape)$>pupa update az --scrape
```

This will throw a `ScrapeError` because we haven't written any of the main scrapers yet, but before it does we'll see that it creates our `Jurisdiction` object, and the `Organization` representing teh Arizona Secretary of State.

```
no pupa_settings on path, using defaults
az (scrape)
  events: {}
  people: {}
  disclosures: {}
Not checking sessions...
13:30:10 INFO pupa: save jurisdiction Arizona as jurisdiction_ocd-jurisdiction-country:us-state:az-government.json
13:30:10 INFO pupa: save organization Office of the Secretary of State, State of Arizona as organization_1e330580-e20b-11e4-a4f5-e90fe0697b56.json
```

# Starting a new scraper

Now it's time to start writing the real meat and potatoes of our scraping code.

## Locate the source of the data

Check out the [Big Board](https://docs.google.com/spreadsheets/d/18-MvVJXg8TkUUNhtBmWoCEPUWEMf7F6-YVV6x7CWrg4/pubhtml) to see which URL you should use to start. Explore the links on that page until you find the data you're looking for.

For this example, we'll look at the Arizona Super PAC list. 

In [1]:
PAC_LIST_URL = "http://apps.azsos.gov/apps/election/cfs/search/SuperPACList.aspx"

## Adding new scrape routines

We're going to add our code to `az/disclosures.py`. 

```
class ArizonaDisclosureScraper(Scraper):
                                           
    def scrape_super_pacs(self):           
        pass                               
                                           
    def scrape(self):                      
        # needs to be implemented          
        yield from self.scrape_super_pacs()
```

When we're through, the `pupa` CLI commands will call the `scrape` command. It's good practice to follow this pattern to break down that command into a series of subroutines, one for each type of data you're returning. The pupa software actually doesn't care, though, it just expects a stream of Open Civic Data scrape objects (`Person`, `Organizaton`, `Event`, etc).

## Developing your scraper

At this point, you might want to move to a REPL (or, even better, to an IPython notebook) so that you can start figuring out how to obtain the target data. You'll 

In this example, things are fairly straightforward.  There's a `<table>` element in the middle of the page that has all the information we need to generate the `Organization` objects that we want.

In [9]:
from lxml import etree
from lxml.html import HTMLParser

import scrapelib

The scraper we'll eventually write is a subclass of `scrapelib.Scraper()` (see the `Scraper` class in [pupa/pupa/scrape/base.py](https://github.com/influence-usa/pupa/blob/disclosures/pupa/scrape/base.py)), and working with the parent class will be close enough to reality when we're undergoing trial and error at first.

In [89]:
my_scraper = scrapelib.Scraper()

In [90]:
resp = my_scraper.urlretrieve(PAC_LIST_URL)

In [13]:
resp

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<!-- saved from url=(0064)http://apps.azsos.gov/apps/election/cfs/search/SuperPACList.aspx -->\n<html xmlns="http://www.w3.org/1999/xhtml"><head id="ctl00_ctl00_PageHead"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><title>\n\tSuperPAC Committees List\n</title>\n    \n<meta http-equiv="Content-Style-Type" content="text/css"><meta name="title" content="Arizona Secretary of State"><meta name="description" content="Secretary of State"><meta name="originatorJurisdiction" content="Arizona Secretary of State"><meta name="govType" content="State Government"><meta name="medium" content="Web page"><meta name="accessConstraints" content="none"><link href="./SuperPACList.aspx_files/apps-style-newazsos.css" rel="stylesheet" type="text/css" media="all"><link href="./SuperPACList.aspx_files/SiteMasterStyles.css" rel="stylesheet" type="text/css" media="all"

In [15]:
d = etree.fromstring(resp, parser=HTMLParser())

The easiest thing to do is just look for the table we're interested by writing an xpath query. The `<table>` 

In [17]:
d.xpath('//table')

[<Element table at 0x7fbe9c435138>, <Element table at 0x7fbe9c435188>]

Hm, looks like there's more than one, so we're going to have to narrow our XPath query. Here's where we can cheat: using Chrome/Chromium, we can right click on the table, and select "Inspect Element" from the dropdown. This opens Developer Tools and shows us where in the page's source that object is. Right click on the `<table>` tag and select "Copy XPath". Now paste the results into a text editor.  What you'll see is:

    //*[@id="ctl00_ctl00_MainPanel"]/table
    
Chrome DevTools does its best to find a short-ish query that uniquely identifies the node you've selected. The results of this query will be more targeted.

In [18]:
d.xpath('//*[@id="ctl00_ctl00_MainPanel"]/table')

[<Element table at 0x7fbe9c435188>]

Notice that this is still a list. To access it we'll need to select the object by indexing with `[0]`.

We know that the information we need is in this table, and using IPython we can test out scripts that scrape the data.

In [21]:
target_table = d.xpath('//*[@id="ctl00_ctl00_MainPanel"]/table')[0]

The `table` element can, in turn, be queried with an XPath query. We can get all of the rows with `tbody/tr`.

In [34]:
rows = target_table.xpath('tbody/tr')
len(rows)

26

In [36]:
rows[0]

<Element tr at 0x7fbe9c43f138>

In [97]:
def find_the_table(html_document):
    return d.xpath('//*[@id="ctl00_ctl00_MainPanel"]/table')[0]

To see what the original HTML string of any element looks like, use:

In [37]:
etree.tostring(rows[5])

b'<tr>\n\n<td>201000081</td>\n<td>Arizona List P.A.C.<br />PO BOX 42294<br />TUCSON, AZ 85733 </td>\n<td>(520) 327-0520</td>\n<td>01/25/2015</td>\n<td>01/24/2019</td>\n</tr>\n\n'

Now we can write functions to query for, and parse each table. It can be helpful when debugging to break up your scraper's code into small functions, so that when (not if ;) something goes wrong, it's easier to tell where and why.

In [91]:
def scrape_table_first_draft(table_element):
    scraped_rows = []
    for row in table_element.xpath('tbody/tr'):
        _data = {}
        columns = row.xpath('td')
        _data['org_id'] = columns[0].text_content()
        _data['org_name_and_address'] = columns[1].text_content()
        _data['org_phone'] = columns[2].text_content()
        _data['org_begin_date'] = columns[3].text_content()
        _data['org_end_date'] = columns[4].text_content()
        scraped_rows.append(_data)
    return scraped_rows

In [92]:
scrape_table_first_draft(target_table)

IndexError: list index out of range

Oops! Let's look at what happened. We got an `IndexError` when looking for item `0` in the list `columns`. Expect to see a lot of these kinds of errors when writing `lxml` scrapers. It usually means that an XPath query returned an empty list.  Let's make use our `"td"` query is working on each row:

In [94]:
[row.xpath("td") for row in target_table.xpath('tbody/tr')]

[[],
 [<Element td at 0x7fbe9c4512c8>,
  <Element td at 0x7fbe9c4518b8>,
  <Element td at 0x7fbe9c451908>,
  <Element td at 0x7fbe9c451958>,
  <Element td at 0x7fbe9c4519a8>],
 [<Element td at 0x7fbe9c4519f8>,
  <Element td at 0x7fbe9c451a48>,
  <Element td at 0x7fbe9c451a98>,
  <Element td at 0x7fbe9c451ae8>,
  <Element td at 0x7fbe9c451b38>],
 [<Element td at 0x7fbe9c451b88>,
  <Element td at 0x7fbe9c451bd8>,
  <Element td at 0x7fbe9c451c28>,
  <Element td at 0x7fbe9c451c78>,
  <Element td at 0x7fbe9c451cc8>],
 [<Element td at 0x7fbe9c451d18>,
  <Element td at 0x7fbe9c451d68>,
  <Element td at 0x7fbe9c451db8>,
  <Element td at 0x7fbe9c451e08>,
  <Element td at 0x7fbe9c451e58>],
 [<Element td at 0x7fbe9c451ea8>,
  <Element td at 0x7fbe9c451ef8>,
  <Element td at 0x7fbe9c451f48>,
  <Element td at 0x7fbe9c451f98>,
  <Element td at 0x7fbe9c450048>],
 [<Element td at 0x7fbe9c450098>,
  <Element td at 0x7fbe9c4500e8>,
  <Element td at 0x7fbe9c450138>,
  <Element td at 0x7fbe9c450188>,
  <E

Ah, the first row has no `td` elements!  Let's look at it again:

In [96]:
etree.tostring(target_table.xpath('tbody/tr')[0])

b'<tr><th>Filer ID</th><th>Name &amp; Address</th> <th>Phone #</th><th>Begin Date</th><th>End Date</th></tr>\n\n'

Of course: the first row is a header row, and only contains `<th>` tags. (Actually, this is bad HTML. The `th` tags should be inside of a `<thead>` tag, but that's not what was on the page). Let's make sure that there are 5 `td` tags when parsing a row.

In [98]:
def scrape_table_second_draft(table_element):
    scraped_rows = []
    for row in table_element.xpath('tbody/tr'):
        _data = {}
        columns = row.xpath('td')
        if len(columns) == 5:
            _data['org_id'] = columns[0].text_content()
            _data['org_name_and_address'] = columns[1].text_content()
            _data['org_phone'] = columns[2].text_content()
            _data['org_begin_date'] = columns[3].text_content()
            _data['org_end_date'] = columns[4].text_content()
            scraped_rows.append(_data)
    return scraped_rows
        

In [99]:
result = scrape_table_second_draft(target_table)

In [100]:
result

[{'org_begin_date': '06/09/2014',
  'org_end_date': '06/08/2018',
  'org_id': '1066',
  'org_name_and_address': 'AEA FUND FOR PUBLIC EDUCATION (FORMERLY) AZ PAC (AZ EDUCATION ASSN PAC)345 E PALM LNPHOENIX, AZ 85004 ',
  'org_phone': '(602) 264-1774'},
 {'org_begin_date': '04/02/2014',
  'org_end_date': '04/01/2018',
  'org_id': '1354',
  'org_name_and_address': 'AFSCME PEOPLE1625 L ST NWWASHINGTON, DC 20036 ',
  'org_phone': '(202) 429-1088'},
 {'org_begin_date': '02/14/2014',
  'org_end_date': '02/13/2018',
  'org_id': '200602733',
  'org_name_and_address': 'ARIZONA CHAMBER POLITICAL ACTION COMMITTEE3200 N CENTRAL AVE, STE 1125PHOENIX, AZ 85012 ',
  'org_phone': '(602) 248-9172'},
 {'org_begin_date': '07/21/2014',
  'org_end_date': '07/20/2018',
  'org_id': '200602848',
  'org_name_and_address': 'ARIZONA FAMILIES UNITED FOR STRONG COMMUNITIES- PROJECT OF SEIU COPE3707 N 7TH ST, STE 100PHOENIX, AZ 85014 ',
  'org_phone': '(602) 279-8016'},
 {'org_begin_date': '01/25/2015',
  'org_end_d

Hm, let's see if we can do a better job with that `org_name_and_address` field. We're going to want to separate them.

In [56]:
rows = target_table.xpath('tbody/tr')
row = rows[7]

In [79]:
cell = row.xpath('td')[1]

In [80]:
etree.tostring(cell)

b'<td>Arizona Pipe Trades 469<br />3109 N 24TH ST<br />PHOENIX, AZ 85016 </td>\n'

The first part of the cell can be extracted using the `text` property. Unlike `text_content`, it will only return the text up until the first `<br/>` tag.

In [84]:
cell.text

'Arizona Pipe Trades 469'

We can take advantage of the `<br />` elements, by using their `tail` properties. This property returns text that follows the element, stopping before the next element encountered.

In [82]:
for br in cell.xpath('br'):
    print('tail: "{}"'.format(br.tail))

tail: "3109 N 24TH ST"
tail: "PHOENIX, AZ 85016 "


In [88]:
def separate_name_and_address(cell):
    name = cell.text
    address = ', '.join([br.tail for br in cell.xpath('br')])
    return name, address

def scrape_table(table_element):
    scraped_rows = []
    for row in table_element.xpath('tbody/tr'):
        _data = {}
        columns = row.xpath('td')
        if len(columns) == 5:
            _data['org_id'] = columns[0].text_content()
            _name, _address = separate_name_and_address(columns[1])
            _data['org_name'] = _name
            _data['org_address'] = _address
            _data['org_phone'] = columns[2].text_content()
            _data['org_begin_date'] = columns[3].text_content()
            _data['org_end_date'] = columns[4].text_content()
            scraped_rows.append(_data)
    return scraped_rows

In [86]:
results = new_scrape_function(target_table)

In [87]:
results

[{'org_address': '345 E PALM LN, PHOENIX, AZ 85004 ',
  'org_begin_date': '06/09/2014',
  'org_end_date': '06/08/2018',
  'org_id': '1066',
  'org_name': 'AEA FUND FOR PUBLIC EDUCATION (FORMERLY) AZ PAC (AZ EDUCATION ASSN PAC)',
  'org_phone': '(602) 264-1774'},
 {'org_address': '1625 L ST NW, WASHINGTON, DC 20036 ',
  'org_begin_date': '04/02/2014',
  'org_end_date': '04/01/2018',
  'org_id': '1354',
  'org_name': 'AFSCME PEOPLE',
  'org_phone': '(202) 429-1088'},
 {'org_address': '3200 N CENTRAL AVE, STE 1125, PHOENIX, AZ 85012 ',
  'org_begin_date': '02/14/2014',
  'org_end_date': '02/13/2018',
  'org_id': '200602733',
  'org_name': 'ARIZONA CHAMBER POLITICAL ACTION COMMITTEE',
  'org_phone': '(602) 248-9172'},
 {'org_address': '3707 N 7TH ST, STE 100, PHOENIX, AZ 85014 ',
  'org_begin_date': '07/21/2014',
  'org_end_date': '07/20/2018',
  'org_id': '200602848',
  'org_name': 'ARIZONA FAMILIES UNITED FOR STRONG COMMUNITIES- PROJECT OF SEIU COPE',
  'org_phone': '(602) 279-8016'},
 {

## Adding to the DisclosureScraper object

After we think that we're able to scrape the data we want, we should add the code to our scraper object.

```
class ArizonaDisclosureScraper(Scraper):

    def scrape_super_pacs(self):
    
        def find_the_table(html_document):
            return d.xpath('//*[@id="ctl00_ctl00_MainPanel"]/table')[0]
    
        def separate_name_and_address(cell):
            name = cell.text
            address = ', '.join([br.tail for br in cell.xpath('br')])
            return name, address
    
        def scrape_table(table_element):
            scraped_rows = []
            for row in table_element.xpath('tbody/tr'):
                _data = {}
                columns = row.xpath('td')
                if len(columns) == 5:
                    _data['org_id'] = columns[0].text_content()
                    _name, _address = separate_name_and_address(columns[1])
                    _data['org_name'] = _name
                    _data['org_address'] = _address
                    _data['org_phone'] = columns[2].text_content()
                    _data['org_begin_date'] = columns[3].text_content()
                    _data['org_end_date'] = columns[4].text_content()
                    scraped_rows.append(_data)
            return scraped_rows
            
        PAC_LIST_URL = "http://apps.azsos.gov/apps/election/cfs/search/SuperPACList.aspx"
        
        resp = self.urlretrieve(PAC_LIST_URL)
        
        html_document = etree.fromstring(resp, parser=HTMLParser())
        
        target_table = find_the_table(html_document)
        
        results = scrape_the_table(target_table)

    def scrape(self):                      
        # needs to be implemented          
        yield from self.scrape_super_pacs()
```

## Use scraped data to build Open Civic Data objects

Let's look at how we can use the PAC data we scraped to build an `Organization` object.

In [101]:
result = results[0]

result

{'org_address': '345 E PALM LN, PHOENIX, AZ 85004 ',
 'org_begin_date': '06/09/2014',
 'org_end_date': '06/08/2018',
 'org_id': '1066',
 'org_name': 'AEA FUND FOR PUBLIC EDUCATION (FORMERLY) AZ PAC (AZ EDUCATION ASSN PAC)',
 'org_phone': '(602) 264-1774'}

If we import the `Organization` scraper model, this becomes very easy:

In [102]:
from pupa.scrape.popolo import Organization

In [103]:
my_org = Organization(
    name=result['org_name'],
    classification='political action committee',
    founding_date=result['org_begin_date'],
    dissolution_date=result['org_end_date']
)

That's actually more information than we needed. Only the `name` and `classification` properties were required.

To add other information, like contact details and identifiers, we can use special helper functions that come with the `Organization` class:

In [105]:
my_org.add_identifier(
    identifier=result['org_id'],
    scheme='urn:az-state:committee'
    )

In [107]:
my_org.add_contact_detail(
    type='address',
    value=result['org_address']
)

In [109]:
my_org.add_contact_detail(
    type='voice',
    value=result['org_phone']
)

To see your organization as a `dict`, use the `as_dict` method:

In [111]:
my_org.as_dict()

{'_id': '102e7fde-e24a-11e4-a4f5-e90fe0697b56',
 'classification': 'political action committee',
 'contact_details': [{'note': '',
   'type': 'address',
   'value': '345 E PALM LN, PHOENIX, AZ 85004 '},
  {'note': '', 'type': 'voice', 'value': '(602) 264-1774'}],
 'dissolution_date': '06/08/2018',
 'extras': {},
 'founding_date': '06/09/2014',
 'identifiers': [{'identifier': '1066', 'scheme': 'urn:az-state:committee'}],
 'image': '',
 'links': [],
 'name': 'AEA FUND FOR PUBLIC EDUCATION (FORMERLY) AZ PAC (AZ EDUCATION ASSN PAC)',
 'other_names': [],
 'parent_id': None,
 'source_identified': False,
 'sources': []}

Ah! That reminds me: We have to be sure to do two things to all `Person` and `Organization` objects returned by our scraper.

First, Make sure `source_identified` is `True`

In [112]:
my_org.source_identified = True

Next, make sure you include the source (ie, the URL that you found the organization at)

In [114]:
my_org.add_source(url=PAC_LIST_URL)

## Yielding your new objects

Adding the new code to our scraper:

```
class ArizonaDisclosureScraper(Scraper):

    def scrape_super_pacs(self):

        def find_the_table(html_document):
            return d.xpath('//*[@id="ctl00_ctl00_MainPanel"]/table')[0]

        def separate_name_and_address(cell):
            name = cell.text
            address = ', '.join([br.tail for br in cell.xpath('br')])
            return name, address

        def scrape_table(table_element):
            scraped_rows = []
            for row in table_element.xpath('tbody/tr'):
                _data = {}
                columns = row.xpath('td')
                if len(columns) == 5:
                    _data['org_id'] = columns[0].text_content()
                    _name, _address = separate_name_and_address(columns[1])
                    _data['org_name'] = _name
                    _data['org_address'] = _address
                    _data['org_phone'] = columns[2].text_content()
                    _data['org_begin_date'] = columns[3].text_content()
                    _data['org_end_date'] = columns[4].text_content()
                    scraped_rows.append(_data)
            return scraped_rows

        PAC_LIST_URL = "http://apps.azsos.gov/apps/election/cfs/search/SuperPACList.aspx"

        resp = self.urlretrieve(PAC_LIST_URL)

        html_document = etree.fromstring(resp, parser=HTMLParser())

        target_table = find_the_table(html_document)

        results = scrape_the_table(target_table)
        
        for result in results:
            _org = Organization(
                name=result['org_name'],
                classification='political action committee',
                founding_date=result['org_begin_date'],
                dissolution_date=result['org_end_date']
            )
            
            _org.add_identifier(
                identifier=result['org_id'],
                scheme='urn:az-state:committee'
            )
            
            _org.add_contact_detail(
                type='address',
                value=result['org_address']
            )
            
            _org.add_contact_detail(
                type='voice',
                value=result['org_phone']
            )
            
            _org.add_source(url=PAC_LIST_URL)
            
            _org.source_identified = True
            
            yield _org
            

    def scrape(self):                      
        # needs to be implemented          
        yield from self.scrape_super_pacs()
```

## Finished! 

You don't need to worry about anything after this point. After doing the hard work of writing your scraper code, you can hand off the rest (validation, database storage, deduplication) to the `pupa` framework.  